## avoided crossing data to extract maximum dissipator frequency $\omega_\mathrm{max}/2\pi$ and couping strength $g$

In [59]:
import h5py
import numpy as np


In [4]:
# read data from Labber directory
labber_dir = 'G:\\Shared drives\\Labber\\Data\\2023\\'
date_dir = '04\\Data_0428\\'
filename = 'diss09_5578_fluxSweep_2.hdf5'

f = h5py.File(labber_dir + date_dir + filename,'r')


In [74]:
s21 = f['Traces']['Agilent Network Analyzer - S21'][()]
numsteps = f['Traces']['Agilent Network Analyzer - S21_N'][0][()]
frange = f['Traces']['Agilent Network Analyzer - S21_t0dt'][0][()]
freqs = np.linspace(frange[0],frange[0]+frange[1]*(numsteps-1),numsteps)
currents = np.array(f['Data']['Data'])

In [73]:
f['Traces']['Agilent Network Analyzer - S21_t0dt'][0]

array([5.5675e+09, 1.5625e+04])

In [75]:
len(freqs)

1601

In [68]:
# fit each trace for frequency
# grab trace from s21 matrix
index = 0
idata = s21[:,0,index]
qdata = s21[:.1,index]
plt.plot


# extract current periodicity and translate current to flux

1601

### model

In [64]:
### model

In [ ]:
def avoided_crossing(x, wq_max, wc, g,phi_offset,d):
    y = np.zeros(len(x))
    for i in range(len(x)):
        flux_dep = np.emath.power(abs(np.cos(np.pi*x[i]-phi_offset)),2) + d**2*np.emath.power(abs(np.sin(np.pi*x[i]-phi_offset)),2)
        wq = wq_max*np.emath.power(flux_dep, 1/4)
        if i<=221: #where the cropped lower curve ends in fitx
            y[i] = 1/2*(-np.emath.sqrt(4*g**2+(wc-wq)**2) + np.emath.sqrt(4*g**2+(wc+wq)**2))
        else:
            y[i] = 1/2*(np.emath.sqrt(4*g**2+(wc-wq)**2) + np.emath.sqrt(4*g**2+(wc+wq)**2))
    return y

def avoided_crossing_lower(x, wq_max, wc, g,phi_offset,d):
    flux_dep = np.emath.power(abs(np.cos(np.pi*x-phi_offset)),2) + d**2*np.emath.power(abs(np.sin(np.pi*x-phi_offset)),2)
    wq = wq_max*np.emath.power(flux_dep, 1/4)
    return 1/2*(-np.emath.sqrt(4*g**2+(wc-wq)**2) + np.emath.sqrt(4*g**2+(wc+wq)**2))

def avoided_crossing_upper(x, wq_max, wc, g, phi_offset,d):
    flux_dep = np.emath.power(abs(np.cos(np.pi*x-phi_offset)),2) + d**2*np.emath.power(abs(np.sin(np.pi*x-phi_offset)),2)
    wq = wq_max*np.emath.power(flux_dep, 1/4)
    return 1/2*(np.emath.sqrt(4*g**2+(wc-wq)**2) + np.emath.sqrt(4*g**2+(wc+wq)**2))
def predict_disspator_frequency(x,wq_max,wc,g,phi_offset,d):
    flux_dep = np.emath.power(abs(np.cos(np.pi*x-phi_offset)),2) + d**2*np.emath.power(abs(np.sin(np.pi*x-phi_offset)),2)
    wq = wq_max*np.emath.power(flux_dep, 1/4)
    return wq

